# ĐỒ ÁN CUỐI KỲ - PHÂN TÍCH DỮ LIỆU BẢO TOÀN TÍNH RIÊNG TƯ

## THÀNH VIÊN NHÓM:

- 22120304 - Nguyễn Thị Kim Quý

- 22120338 - Đỗ Hạnh Thảo

- 22120352 - Phạm Nguyễn Quang Thoại

In [36]:
import pandas as pd
import numpy as np
import re

### 1. Đọc file

In [37]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("Số dòng trong train.csv", len(train_data))
print(f"\nSố cột: {len(train_data.columns)}")
print("Tên các cột:", list(train_data.columns))
print("\n 5 dòng đầu trong train.csv:\n")
print(train_data.head())

Số dòng trong train.csv 1317

Số cột: 20
Tên các cột: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong train.csv:

        Make                                    Model    Price  Year  \
0  Ssangyong                               Rexton RX7   975000  2013   
1    Hyundai  Creta SX (O) 1.5 Petrol CVT [2020-2022]  1748999  2022   
2       Audi                     A4 2.0 TDI (143 bhp)  1150000  2012   
3    Hyundai        Grand i10 Magna AT 1.2 Kappa VTVT   549000  2018   
4    Hyundai                       Elite i20 Asta 1.2   675000  2017   

   Kilometer Fuel Type Transmission   Location   Color   Owner Seller Type  \
0      72000    Diesel    Automatic  Bangalore  Silver   First  Individual   
1       2670    Petrol    Automatic    Kolkata   White   First  Individual   
2   

In [38]:
print("Số dòng trong test.csv", len(test_data))
print(f"\nSố cột: {len(test_data.columns)}")
print("Tên các cột:", list(test_data.columns))
print("\n 5 dòng đầu trong test.csv:\n")
print(test_data.head())

Số dòng trong test.csv 330

Số cột: 20
Tên các cột: ['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong test.csv:

            Make                           Model    Price  Year  Kilometer  \
0           Ford      Endeavour Trend 2.2 4x2 AT  2350000  2016      75000   
1         Toyota  Urban Cruiser Premium Grade AT  1050000  2021       1910   
2  Maruti Suzuki                    Alto 800 Lxi   210000  2014      42505   
3          Honda                  City SV Diesel   550000  2014      85000   
4        Hyundai                       Eon Era +   290000  2018      70000   

  Fuel Type Transmission  Location   Color   Owner Seller Type   Engine  \
0    Diesel    Automatic    Mohali   White  Second  Individual  2198 cc   
1    Petrol    Automatic  Varanasi   White   First  Ind

### 2. Tiền xử lý dữ liệu

In [39]:
# Hàm trích xuất số từ chuỗi
def extract_number(s):
    if pd.isna(s):
        return np.nan
    match = re.search(r'(\d+)', str(s).replace(',', ''))
    if match:
        return int(match.group(1))
    return np.nan

# Hàm label encode
# def label_encode(series, mapping=None):
#     if mapping is None:
#         uniques = series.dropna().unique()
#         mapping = {k: v for v, k in enumerate(sorted(uniques))}
#     encoded = series.map(mapping)
#     return encoded, mapping
def label_encode(series):
    uniques = series.dropna().unique()
    mapping = {k: v for v, k in enumerate(sorted(uniques))}
    encoded = series.map(mapping)
    return encoded, mapping

In [40]:
# Bước 1: Trích số từ các cột chuỗi số
for col in ['Engine', 'Max Power', 'Max Torque']:
    train_data[col] = train_data[col].apply(extract_number)
    test_data[col] = test_data[col].apply(extract_number)

# Bước 2: Label encoding thủ công cho các cột phân loại
categorical_cols = ['Make', 'Model', 'Fuel Type', 'Transmission', 'Location', 
                    'Color', 'Owner', 'Seller Type', 'Drivetrain']
mappings = {}

for col in categorical_cols:
    train_data[col], mappings[col] = label_encode(train_data[col])
    test_data[col] = test_data[col].map(mappings[col])

# Bước 3: Xử lý giá trị thiếu
# Với categorical NaN -> -1
for col in categorical_cols:
    train_data[col] = train_data[col].fillna(-1)
    test_data[col] = test_data[col].fillna(-1)

# Với numeric NaN -> trung bình cột train
numeric_cols = ['Year', 'Kilometer', 'Engine', 'Max Power', 'Max Torque', 
                'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']
for col in numeric_cols:
    mean = train_data[col].mean()
    train_data[col] = train_data[col].fillna(mean)
    test_data[col] = test_data[col].fillna(mean)

# Bước 4: Chuẩn hóa dữ liệu numeric
for col in numeric_cols:
    mean = train_data[col].mean()
    std = train_data[col].std()
    if std == 0:
        std = 1e-8  # tránh chia 0
    train_data[col] = (train_data[col] - mean) / std
    test_data[col] = (test_data[col] - mean) / std

# Bước 5: Tách X và Y
y_train = train_data['Price']
X_train = train_data.drop(columns=['Price'])
y_test = test_data['Price']
X_test = test_data.drop(columns=['Price'])

print("Sau khi tiển xử lý dữ liệu cho train_data: ")

print("Train.shape = ", X_train.shape)
print("Các cột trong X_train:", list(X_train.columns))
print("\n 5 dòng đầu trong X_train đã xử lý:\n")
print(X_train.head())

Sau khi tiển xử lý dữ liệu cho train_data: 
Train.shape =  (1317, 19)
Các cột trong X_train: ['Make', 'Model', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong X_train đã xử lý:

   Make  Model      Year  Kilometer  Fuel Type  Transmission  Location  Color  \
0    25    538 -0.989540   0.274506          1             0         6     14   
1     7    227  1.644419  -0.817523          5             0        33     15   
2     0     33 -1.282202  -0.229044          1             0        53      2   
3     7    397  0.473770  -0.024767          5             0        24     15   
4     7    295  0.181108  -0.094214          5             1        24     15   

   Owner  Seller Type        Engine     Max Power    Max Torque  Drivetrain  \
0      0            2  1.503194e+00  8.338177e-01  1.095560e+00     

In [41]:
print("Sau khi tiền xử lý dữ liệu cho test_data:")
print("Test.shape = ", X_test.shape)
print("Các cột trong X_test:", list(X_test.columns))
print("\n 5 dòng đầu trong X_test đã xử lý:\n")
print(X_test.head())

Sau khi tiền xử lý dữ liệu cho test_data:
Test.shape =  (330, 19)
Các cột trong X_test: ['Make', 'Model', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity']

 5 dòng đầu trong X_test đã xử lý:

   Make  Model      Year  Kilometer  Fuel Type  Transmission  Location  Color  \
0   5.0   -1.0 -0.111554   0.321759        1.0             0      41.0     15   
1  27.0   -1.0  1.351757  -0.829494        5.0             0      68.0     15   
2  17.0   56.0 -0.696878  -0.190075        5.0             1      54.0     13   
3   6.0  172.0 -0.696878   0.479271        1.0             1      26.0     14   
4   7.0  320.0  0.473770   0.243003        5.0             1      32.0     15   

   Owner  Seller Type    Engine  Max Power  Max Torque  Drivetrain    Length  \
0    2.0            2  0.744884   0.430370    0.974486         2.0  1.369

### 3.1 - 3.2: Xây dựng mô hình và huấn luyện

Mô hình: Linear Regression

Thuật toán tối ưu: Stochastic Gradient Descent

**Các cột dữ liệu được chọn**:

- Year: Năm sản xuất ảnh hưởng đến độ mới, giá trị còn lại.

- Kilometer: Số km đã đi là chỉ số hao mòn – ảnh hưởng trực tiếp đến giá.

- Fuel Type: Xe chạy xăng, dầu hay điện có mức giá khác nhau.

- Transmission: Xe số sàn thường rẻ hơn xe số tự động.

- Engine: Dung tích/mã lực động cơ ảnh hưởng đến giá xe.

- Max Power: Công suất phản ánh hiệu năng, thường đi kèm giá.

- Seating Capacity: Xe 4 chỗ, 7 chỗ, 9 chỗ có giá khác nhau.

- Fuel Tank Capacity: Phụ nhưng vẫn phản ánh phần nào kích thước/loại xe.

**Các công thức hồi quy sẽ sử dụng**:

- **y = a1x1 + a2x2 + a3x3 + a4x4**: Đây là mô hình hồi quy tuyến tính đơn giản, chuẩn, rất ổn cho baseline.

- **y = a1x1^2 + a2x2 + a3x3^2 + a4x4**: Đây là kỹ thuật polynomial features, giúp mô hình có thể nắm bắt được quan hệ phi tuyến.

- **y = a1(x1 + x2) + a3x3^ + a4x4**: Có các tương tác tiềm năng sau:

    - Year + Kilometer: Xe mới + số kilomet đã chạy ít -> giá cao

    - Make và Model: kết hợp thương hiệu + dòng xe đặc biệt -> đắt tiền.

    - Fuel Type và Engine / Max Power / Max Torque: Xe điện có thể có cấu hình động cơ và công suất khác biệt với xe xăng/dầu, tương tác giữa loại nhiên liệu và hiệu năng động cơ có thể quan trọng.

    - Seating Capacity và Fuel Tank Capacity: Xe nhiều chỗ thường có bình nhiên liệu lớn hơn, tương tác giúp phân biệt phân khúc xe.
    
    - Transmission và DriveTrain: Ví dụ xe số tự động + hệ dẫn động 4 bánh có thể giá cao hơn số sàn + dẫn động cầu trước.

- **y = a1x1x2 + a3x3^2**:
Cũng có các tương tác như trên

- **Regularized Linear Regression: Ridge Regression (L2 Regularization)**: λ∑(ai^2): Tránh overfitting

In [42]:
X_train = X_train.drop(columns=['Color', 'Length', 'Width', 'Height'])
X_test = X_test.drop(columns=['Color', 'Length', 'Width', 'Height'])

print("Columns in X_train:", list(X_train.columns))
print("Columns in X_test:", list(X_test.columns))

Columns in X_train: ['Make', 'Model', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Seating Capacity', 'Fuel Tank Capacity']
Columns in X_test: ['Make', 'Model', 'Year', 'Kilometer', 'Fuel Type', 'Transmission', 'Location', 'Owner', 'Seller Type', 'Engine', 'Max Power', 'Max Torque', 'Drivetrain', 'Seating Capacity', 'Fuel Tank Capacity']


In [43]:
# Tính mean và std của y_train
y_mean = y_train.mean()
y_std = y_train.std()

# Chuẩn hóa y_train
y_train_scaled = (y_train - y_mean) / y_std

In [44]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def sgd_linear_regression(X, y, transform_fn, lr=1e-5, max_epoch=500, lambda_l2=0.0):
    n_samples = X.shape[0]
    X_transformed = transform_fn(X)
    n_features = X_transformed.shape[1]
    weights = np.zeros(n_features)
    
    print("X_transformed max, min:", np.max(X_transformed), np.min(X_transformed))
    print("Any NaN in X_transformed?", np.any(np.isnan(X_transformed)))
    print("Any Inf in X_transformed?", np.any(np.isinf(X_transformed)))


    for epoch in range(max_epoch):
        indices = np.arange(n_samples)
        np.random.shuffle(indices)  
        for i in indices:
            xi = X_transformed[i]
            yi = y[i]
            y_pred = np.dot(weights, xi)
            error = y_pred - yi
            gradient = error * xi + 2 * lambda_l2 * weights
            
            # Check NaN or Inf
            if np.any(np.isnan(gradient)) or np.any(np.isinf(gradient)):
                print(f"Warning: gradient NaN/Inf at epoch {epoch}, sample {i}")
                continue
            
            weights_new = weights - lr * gradient
            if np.any(np.isnan(weights_new)) or np.any(np.isinf(weights_new)):
                print(f"Warning: weights overflow at epoch {epoch}, sample {i}")
                continue
            weights = weights_new
            
        if epoch % 10 == 0:
            y_pred_scaled = np.dot(X_transformed, weights)
            # Chuyển dự đoán y_pred_scaled về thang đo gốc trước khi tính mse
            y_pred_original = y_pred_scaled * y_std + y_mean
            mse = mean_squared_error(y, y_pred_original)
            print(f"Epoch {epoch}: MSE = {mse:.4f}")

    y_pred_scaled = np.dot(X_transformed, weights)
    mse = mean_squared_error(y, y_pred_scaled * y_std + y_mean)
    return weights, mse

In [45]:
def formula_1(X):  
    return X[:, [0, 1, 2, 3]]

def formula_2(X):  
    return np.column_stack((X[:,0]**2, X[:,1], X[:,2]**2, X[:,3]))

def formula_3(X):  # y = a1(x1 + x2) + a3x3^2 + a4x4
    x1_plus_x2 = X[:,0] + X[:,1]
    return np.column_stack((x1_plus_x2, X[:,2]**2, X[:,3]))

def formula_4(X):  # y = a1*x1*x2 + a3*x3^2
    x1x2 = X[:,0] * X[:,1]
    return np.column_stack((x1x2, X[:,2]**2))

def formula_5_ridge(X):  # Ridge Regression: y = a1x1 + a2x2 + ... + a15x15
    return X  # Giữ nguyên toàn bộ 15 đặc trưng


def run_all_formulas(X_train, y_train):
    formulas = [formula_1, formula_2, formula_3, formula_4, formula_5_ridge]
    names = [
        "Công thức 1: y = a1x1 + a2x2 + a3x3 + a4x4",
        "Công thức 2: y = a1x1^2 + a2x2 + a3x3^2 + a4x4",
        "Công thức 3: y = a1(x1 + x2) + a3x3^2 + a4x4",
        "Công thức 4: y = a1x1x2 + a3x3^2",
        "Công thức 5: Ridge Regression (15 cột)"
    ]

    for i, formula in enumerate(formulas):
        lambda_l2 = 0.1 if i == 4 else 0.0
        weights, mse = sgd_linear_regression(X_train, y_train, formula, lambda_l2=lambda_l2)
        print(f"{names[i]}")
        print(f"   ➤ MSE: {mse:.2f}")
        print(f"   ➤ Weights: {weights}\n")

In [46]:
run_all_formulas(X_train.values, y_train_scaled.values)

X_transformed max, min: 805.0 -8.306094180508147
Any NaN in X_transformed? False
Any Inf in X_transformed? False
Epoch 0: MSE = 11920360112777509187360906201843026554850223653888032326831200548173756145480772332686180739984164564019127530845437512669674340352.0000


C:\Users\Admin\AppData\Local\Temp\ipykernel_10428\2748322410.py:23: RuntimeWarning: overflow encountered in multiply
  gradient = error * xi + 2 * lambda_l2 * weights
C:\Users\Admin\AppData\Local\Temp\ipykernel_10428\2748322410.py:39: RuntimeWarning: overflow encountered in multiply
  y_pred_original = y_pred_scaled * y_std + y_mean
C:\Users\Admin\AppData\Local\Temp\ipykernel_10428\2748322410.py:2: RuntimeWarning: overflow encountered in square
  return np.mean((y_true - y_pred) ** 2)


Epoch 10: MSE = inf
Epoch 20: MSE = inf
Epoch 30: MSE = inf
Epoch 40: MSE = inf
Epoch 50: MSE = inf
Epoch 60: MSE = inf
Epoch 70: MSE = inf
Epoch 80: MSE = inf
Epoch 90: MSE = inf
Epoch 100: MSE = inf
Epoch 110: MSE = inf
Epoch 120: MSE = inf
Epoch 130: MSE = inf
Epoch 140: MSE = inf
Epoch 150: MSE = inf
Epoch 160: MSE = inf
Epoch 170: MSE = inf
Epoch 180: MSE = inf
Epoch 190: MSE = inf
Epoch 200: MSE = inf
Epoch 210: MSE = inf
Epoch 220: MSE = inf
Epoch 230: MSE = inf
Epoch 240: MSE = inf
Epoch 250: MSE = inf
Epoch 260: MSE = inf
Epoch 270: MSE = inf
Epoch 280: MSE = inf
Epoch 290: MSE = inf


KeyboardInterrupt: 

### 3.3 Thêm nhiễu